## Import Modules

In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
import os
from os import listdir
from os.path import isfile, join
import time
import math
from IPython.display import clear_output
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import matplotlib.pyplot as plt
import torchvision.transforms as T
#from torchsampler import ImbalancedDatasetSampler
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
print('cuda available: '+ str(torch.cuda.is_available()))
#from skimage.morphology import disk, binary_dilation

from models_avm import NestedUNet
from loss_fun_avm import compute_per_channel_dice, DiceLoss, FocalLoss
from tra_val_avm import train, validation
from data_loader_avm import Dataset

cuda available: True


## Data Path

In [2]:
path_tra1 = '/work/samhong833/Data_AVM/forUNetpp/1_tra/1'
list_tra1 = os.listdir(path_tra1)
for i in range(len(list_tra1)):
    list_tra1[i] = path_tra1+'/'+list_tra1[i]
    
list_tra = list_tra1
    
path_val1 = '/work/samhong833/Data_AVM/forUNetpp/2_val/1'
list_val1 = os.listdir(path_val1)
for i in range(len(list_val1)):
    list_val1[i] = path_val1+'/'+list_val1[i]
    
list_val = list_val1

path_ts1 = '/work/samhong833/Data_AVM/forUNetpp/3_ts/1'
list_ts1 = os.listdir(path_ts1)
for i in range(len(list_ts1)):
    list_ts1[i] = path_ts1+'/'+list_ts1[i]
    
list_ts = list_ts1

path_tra_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/1_tra'
path_val_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/2_val'
path_ts_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/3_ts'

## Data Loader

In [3]:
train_data = Dataset(list_tra,path_tra_lab_txt,rand_dilate=True,max_dilate_factor=40)
train_loader = torch.utils.data.DataLoader(
    train_data, 
    batch_size = 12,
    shuffle = True,   
)

val_data = Dataset(list_val,path_val_lab_txt,rand_dilate=True,max_dilate_factor=40)
val_loader = torch.utils.data.DataLoader(
    dataset = val_data,           
    batch_size = 4,                 
    shuffle = False,              
)

## Initialize Model and Optimization Parameters

In [4]:
# call model cuda for gpu
model = NestedUNet().cuda()

# define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.95)

# loss function
kwargs = {"alpha": 0.5, "gamma": 3, "reduction": 'mean'}
criterion_FL = FocalLoss(**kwargs)
criterion_DICE = DiceLoss()
loss = [criterion_FL,criterion_DICE]

## Start to Train

In [ ]:
# Create Directory
path = '/work/samhong833/Models/Seg_dia40'

if os.path.isdir(path)==False:
    os.mkdir(path)
path = os.path.join(path,"train")
if os.path.isdir(path)==False:
    os.mkdir(path)       
filenum = glob.glob(path + "/exp*")
path = path + "/exp" + str(len(filenum)+1)
os.mkdir(path)

# Train the Model
epochs = 500 # The number of epochs

valloss = 0
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(path=path,
          model=model,
          loss=loss,
          optimizer=optimizer,
          dataloader=train_loader,
          epoch=epoch,
          scheduler=scheduler)
    print('-' * 89)
    vallossnew = validation(path=path,
          model=model,
          loss=loss,
          dataloader=val_loader,
          epoch=epoch)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s'.format(epoch, (time.time() - epoch_start_time)))
    print('-' * 89)
    
    scheduler.step()
    
    f2 = open(path + '/model_info.txt', 'a')
    if vallossnew<valloss or epoch ==1: 
        fname = path + '/best_val'  + '.tar'
        torch.save(model.state_dict(), fname)
        valloss = vallossnew
        f2.write('| best_val | epoch {:3d}| '.format(epoch)+'\r\n')        
    f2.close()
    

| epoch   1 |   100/  290 batches | lr 0.0001000 | ms/batch 120383.12 | loss_fl 0.002373 | loss_dice  0.33 | 
| epoch   1 |   200/  290 batches | lr 0.0001000 | ms/batch 116835.87 | loss_fl 0.001520 | loss_dice  0.28 | 
| epoch   1 |   290/  290 batches | lr 0.0001000 | ms/batch 104988.96 | loss_fl 0.001240 | loss_dice  0.26 | 
-----------------------------------------------------------------------------------------
| epoch   1 |   100/  118 batches | | ms/batch 17528.80 | val_loss_fl 0.000794 | val_loss_dice  0.21 | 
| epoch   1 |   118/  118 batches | | ms/batch 3142.96 | val_loss_fl 0.000788 | val_loss_dice  0.21 | 
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 362.89s
-----------------------------------------------------------------------------------------
| epoch   2 |   100/  290 batches | lr 0.0001000 | ms/batch 118568.03 | loss_fl 0.000529 | loss_dice  0.19 | 
| epoch   2 |   200/  290 batches | lr 0.0001000

| epoch  11 |   100/  290 batches | lr 0.0000950 | ms/batch 113417.50 | loss_fl 0.000308 | loss_dice  0.11 | 
| epoch  11 |   200/  290 batches | lr 0.0000950 | ms/batch 113540.54 | loss_fl 0.000312 | loss_dice  0.11 | 
| epoch  11 |   290/  290 batches | lr 0.0000950 | ms/batch 103818.78 | loss_fl 0.000315 | loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| epoch  11 |   100/  118 batches | | ms/batch 21629.43 | val_loss_fl 0.000603 | val_loss_dice  0.13 | 
| epoch  11 |   118/  118 batches | | ms/batch 3862.00 | val_loss_fl 0.000603 | val_loss_dice  0.13 | 
-----------------------------------------------------------------------------------------
| end of epoch  11 | time: 356.27s
-----------------------------------------------------------------------------------------
| epoch  12 |   100/  290 batches | lr 0.0000950 | ms/batch 116946.07 | loss_fl 0.000299 | loss_dice  0.11 | 
| epoch  12 |   200/  290 batches | lr 0.0000950

| epoch  21 |   100/  290 batches | lr 0.0000902 | ms/batch 117185.54 | loss_fl 0.000233 | loss_dice  0.09 | 
| epoch  21 |   200/  290 batches | lr 0.0000902 | ms/batch 118488.88 | loss_fl 0.000227 | loss_dice  0.09 | 
| epoch  21 |   290/  290 batches | lr 0.0000902 | ms/batch 105074.97 | loss_fl 0.000226 | loss_dice  0.09 | 
-----------------------------------------------------------------------------------------
| epoch  21 |   100/  118 batches | | ms/batch 16791.65 | val_loss_fl 0.000534 | val_loss_dice  0.12 | 
| epoch  21 |   118/  118 batches | | ms/batch 3023.11 | val_loss_fl 0.000534 | val_loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| end of epoch  21 | time: 360.57s
-----------------------------------------------------------------------------------------
| epoch  22 |   100/  290 batches | lr 0.0000902 | ms/batch 110955.13 | loss_fl 0.000215 | loss_dice  0.09 | 
| epoch  22 |   200/  290 batches | lr 0.0000902

| epoch  31 |   100/  290 batches | lr 0.0000857 | ms/batch 116252.18 | loss_fl 0.000177 | loss_dice  0.07 | 
| epoch  31 |   200/  290 batches | lr 0.0000857 | ms/batch 118944.33 | loss_fl 0.000172 | loss_dice  0.07 | 
| epoch  31 |   290/  290 batches | lr 0.0000857 | ms/batch 105575.04 | loss_fl 0.000173 | loss_dice  0.07 | 
-----------------------------------------------------------------------------------------
| epoch  31 |   100/  118 batches | | ms/batch 21253.83 | val_loss_fl 0.000702 | val_loss_dice  0.12 | 
| epoch  31 |   118/  118 batches | | ms/batch 4032.34 | val_loss_fl 0.000695 | val_loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| end of epoch  31 | time: 366.08s
-----------------------------------------------------------------------------------------
| epoch  32 |   100/  290 batches | lr 0.0000857 | ms/batch 116664.14 | loss_fl 0.000172 | loss_dice  0.07 | 
| epoch  32 |   200/  290 batches | lr 0.0000857

| epoch  41 |   100/  290 batches | lr 0.0000815 | ms/batch 117710.17 | loss_fl 0.000141 | loss_dice  0.06 | 
| epoch  41 |   200/  290 batches | lr 0.0000815 | ms/batch 118194.56 | loss_fl 0.000138 | loss_dice  0.06 | 
| epoch  41 |   290/  290 batches | lr 0.0000815 | ms/batch 106194.40 | loss_fl 0.000138 | loss_dice  0.06 | 
-----------------------------------------------------------------------------------------
| epoch  41 |   100/  118 batches | | ms/batch 21297.77 | val_loss_fl 0.001153 | val_loss_dice  0.12 | 
| epoch  41 |   118/  118 batches | | ms/batch 3922.57 | val_loss_fl 0.001130 | val_loss_dice  0.12 | 
-----------------------------------------------------------------------------------------
| end of epoch  41 | time: 367.33s
-----------------------------------------------------------------------------------------
| epoch  42 |   100/  290 batches | lr 0.0000815 | ms/batch 117290.78 | loss_fl 0.000125 | loss_dice  0.06 | 
| epoch  42 |   200/  290 batches | lr 0.0000815

...

| epoch 161 |   100/  290 batches | lr 0.0000440 | ms/batch 119496.55 | loss_fl 0.000024 | loss_dice  0.01 | 
| epoch 161 |   200/  290 batches | lr 0.0000440 | ms/batch 118983.40 | loss_fl 0.000024 | loss_dice  0.01 | 
| epoch 161 |   290/  290 batches | lr 0.0000440 | ms/batch 107959.33 | loss_fl 0.000024 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 161 |   100/  118 batches | | ms/batch 23850.20 | val_loss_fl 0.003299 | val_loss_dice  0.13 | 
| epoch 161 |   118/  118 batches | | ms/batch 4220.94 | val_loss_fl 0.003255 | val_loss_dice  0.12 | 
-----------------------------------------------------------------------------------------
| end of epoch 161 | time: 374.52s
-----------------------------------------------------------------------------------------
| epoch 162 |   100/  290 batches | lr 0.0000440 | ms/batch 121309.99 | loss_fl 0.000023 | loss_dice  0.01 | 
| epoch 162 |   200/  290 batches | lr 0.0000440

| epoch 171 |   100/  290 batches | lr 0.0000418 | ms/batch 119747.99 | loss_fl 0.000021 | loss_dice  0.01 | 
| epoch 171 |   200/  290 batches | lr 0.0000418 | ms/batch 117860.90 | loss_fl 0.000021 | loss_dice  0.01 | 
| epoch 171 |   290/  290 batches | lr 0.0000418 | ms/batch 105400.65 | loss_fl 0.000022 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 171 |   100/  118 batches | | ms/batch 23385.39 | val_loss_fl 0.003570 | val_loss_dice  0.12 | 
| epoch 171 |   118/  118 batches | | ms/batch 3973.95 | val_loss_fl 0.003534 | val_loss_dice  0.12 | 
-----------------------------------------------------------------------------------------
| end of epoch 171 | time: 370.37s
-----------------------------------------------------------------------------------------
| epoch 172 |   100/  290 batches | lr 0.0000418 | ms/batch 117893.73 | loss_fl 0.000021 | loss_dice  0.01 | 
| epoch 172 |   200/  290 batches | lr 0.0000418

| epoch 181 |   100/  290 batches | lr 0.0000397 | ms/batch 109913.75 | loss_fl 0.000020 | loss_dice  0.01 | 
| epoch 181 |   200/  290 batches | lr 0.0000397 | ms/batch 108380.85 | loss_fl 0.000020 | loss_dice  0.01 | 
| epoch 181 |   290/  290 batches | lr 0.0000397 | ms/batch 97187.05 | loss_fl 0.000020 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 181 |   100/  118 batches | | ms/batch 18938.57 | val_loss_fl 0.003425 | val_loss_dice  0.13 | 
| epoch 181 |   118/  118 batches | | ms/batch 3172.15 | val_loss_fl 0.003381 | val_loss_dice  0.12 | 
-----------------------------------------------------------------------------------------
| end of epoch 181 | time: 337.60s
-----------------------------------------------------------------------------------------
| epoch 182 |   100/  290 batches | lr 0.0000397 | ms/batch 106919.55 | loss_fl 0.000019 | loss_dice  0.01 | 
| epoch 182 |   200/  290 batches | lr 0.0000397 

| epoch 191 |   100/  290 batches | lr 0.0000377 | ms/batch 113482.13 | loss_fl 0.000018 | loss_dice  0.01 | 
| epoch 191 |   200/  290 batches | lr 0.0000377 | ms/batch 112932.41 | loss_fl 0.000019 | loss_dice  0.01 | 
| epoch 191 |   290/  290 batches | lr 0.0000377 | ms/batch 102937.52 | loss_fl 0.000019 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 191 |   100/  118 batches | | ms/batch 20305.78 | val_loss_fl 0.003681 | val_loss_dice  0.12 | 
| epoch 191 |   118/  118 batches | | ms/batch 3431.80 | val_loss_fl 0.003659 | val_loss_dice  0.12 | 
-----------------------------------------------------------------------------------------
| end of epoch 191 | time: 353.09s
-----------------------------------------------------------------------------------------
| epoch 192 |   100/  290 batches | lr 0.0000377 | ms/batch 112154.11 | loss_fl 0.000018 | loss_dice  0.01 | 
| epoch 192 |   200/  290 batches | lr 0.0000377

| epoch 201 |   100/  290 batches | lr 0.0000358 | ms/batch 115587.89 | loss_fl 0.000018 | loss_dice  0.01 | 


# Log ^